In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('../artifacts/data_ingestion/train.csv')

In [3]:
pd.set_option("display.max_columns", 120)
pd.set_option("display.max_rows", 120)
pd.set_option('display.float_format', '{:.2f}'.format)